<a href="https://colab.research.google.com/github/fatih-crypto/MLOPS/blob/main/person_detection_git_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.1/873.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.5
    Uninstalling tqdm-4.66.5:
      Successfully uninstalled tqdm-4.66.5
  Attempting uninstall: av
    Found existing installation: av 13.0.0
    Uninstalling av-13.0.0:
      Successfully uninstalled av-13.0.0


In [6]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 13 05:33:01 2024
@author: MONSTER
"""
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from ultralytics import YOLO
from tqdm import tqdm
import random
import av

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def extract_points(data):
    return [(shape['points'][0][0], shape['points'][0][1], shape['label'])
            for shape in data['shapes'] if shape['shape_type'] == 'point']

def find_homography(src_points, dst_points):
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)
    M, _ = cv2.findHomography(src_points, dst_points)
    return M

def transform_point(point, M):
    point = np.array([point[0], point[1], 1])
    transformed_point = np.dot(M, point)
    transformed_point = transformed_point / transformed_point[2]
    return transformed_point[:2]

# Load YOLO model
model = YOLO('/content/models/our_model.pt')
# Load JSON data for homography
top_view = load_json('/content/kamera_acilari/kus_bakisi.json')
side_view = load_json('/content/kamera_acilari/kamera_yan.json')

# Extract points and calculate homography matrix
top_points = extract_points(top_view)
side_points = extract_points(side_view)
common_points = [(p1, p2) for p1 in side_points for p2 in top_points if p1[2] == p2[2]]
H = find_homography([p[0][:2] for p in common_points], [p[1][:2] for p in common_points])

# Load the original top-down image
top_down_image = cv2.imread('/content/kamera_acilari/kus_bakisi.png')
top_down_image = cv2.cvtColor(top_down_image, cv2.COLOR_BGR2RGB)

# Open input video file using PyAV
input_video_path = '/content/videoes/MEDIA_~1.TS'
container = av.open(input_video_path)
video_stream = container.streams.video[0]

# Video settings
input_width = video_stream.width
input_height = video_stream.height
fps = float(video_stream.average_rate) if video_stream.average_rate else 30.0
fps = 30.0 if fps <= 0 or np.isinf(fps) or np.isnan(fps) else fps
total_frames = video_stream.frames or None

print(f"Video properties: {input_width}x{input_height} @{fps}fps")

# Lists to store processed data
all_car_points = []
all_transformed_points = []
all_frames = []
all_car_ids = []

# Dictionary to store car colors
car_colors = {}

# Process all frames
for frame in tqdm(container.decode(video=0), total=total_frames, desc="Processing frames"):
    # Convert frame to numpy array
    img = cv2.cvtColor(np.array(frame.to_image()), cv2.COLOR_RGB2BGR)

    # Resize frame if necessary
    if img.shape[:2] != (input_height, input_width):
        img = cv2.resize(img, (input_width, input_height))

    # Predict and track with YOLOv8
    model.overrides['imgsz'] = (1080, 1920)  # Set input size to 1024x1024

    results = model.track(img, persist=True, imgsz=(1080, 1920), tracker="/content/track_algorithm/botsort.yaml")

    car_points = []
    car_ids = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            track_id = int(box.id[0]) if box.id is not None else None

            if model.names[class_id] == 'Person' or model.names[class_id] == 'bus'   and track_id is not None:
                # Get bottom center point of bounding box
                bottom_center = ((x1 + x2) // 2, y2)
                car_points.append(bottom_center)
                car_ids.append(track_id)

                # Assign a color to this car if it doesn't have one
                if track_id not in car_colors:
                    car_colors[track_id] = (random.random(), random.random(), random.random())

                color = [int(c * 255) for c in car_colors[track_id]]

                # Draw bounding box and bottom center point
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.circle(img, bottom_center, 5, color, -1)
                cv2.putText(img, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Transform car points to top view
    transformed_points = [transform_point(p, H) for p in car_points]

    all_car_points.append(car_points)
    all_transformed_points.append(transformed_points)
    all_frames.append(img)
    all_car_ids.append(car_ids)

container.close()

print("All frames processed. Creating visualization...")

# Create figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))

# Initialize plots
top_down_plot = ax1.imshow(top_down_image)
ax1.set_title("Top-Down View with Car Movement", fontsize=16)
ax1.axis('off')

side_view_plot = ax2.imshow(np.zeros((input_height, input_width, 3), dtype=np.uint8))
ax2.set_title("Side View with Car Detection", fontsize=16)
ax2.axis('off')

car_scatter = ax1.scatter([], [], c=[], s=50)
trails = {}

def animate(frame_num):
    # Update scatter plot
    if all_transformed_points[frame_num]:
        x = [p[0] for p in all_transformed_points[frame_num]]
        y = [p[1] for p in all_transformed_points[frame_num]]
        colors = [car_colors[id] for id in all_car_ids[frame_num]]
        car_scatter.set_offsets(np.c_[x, y])
        car_scatter.set_color(colors)

        # Update trails
        for id, point in zip(all_car_ids[frame_num], all_transformed_points[frame_num]):
            if id not in trails:
                trails[id], = ax1.plot([], [], c=car_colors[id], linewidth=2, alpha=0.5)
            trail = trails[id]
            x_data, y_data = trail.get_data()
            x_data = np.append(x_data, point[0])
            y_data = np.append(y_data, point[1])
            trail.set_data(x_data, y_data)

        # Add car IDs as text annotations
        #

    # Update side view image
    side_view_plot.set_array(all_frames[frame_num])

    return car_scatter, side_view_plot, *trails.values()

# Create animation
anim = FuncAnimation(fig, animate, frames=len(all_frames), interval=1000/fps, blit=True)

print("Saving animation...")
anim.save('people_movement_botsort.gif', writer='pillow', fps=fps)
print("Animation saved: people_movement_botsort.gif")

plt.tight_layout()
plt.show()

cv2.destroyAllWindows()
print("Process completed.")

Video properties: 1920x1080 @15.0fps


Processing frames: 0it [00:00, ?it/s]


WARNING ⚠️ imgsz=[1080, 1920] must be multiple of max stride 32, updating to [1088, 1920]
0: 1088x1920 6 Persons, 15526.4ms
Speed: 71.5ms preprocess, 15526.4ms inference, 1.1ms postprocess per image at shape (1, 3, 1088, 1920)


Processing frames: 1it [00:19, 19.17s/it]


WARNING ⚠️ imgsz=[1080, 1920] must be multiple of max stride 32, updating to [1088, 1920]
0: 1088x1920 6 Persons, 12797.0ms
Speed: 19.3ms preprocess, 12797.0ms inference, 0.4ms postprocess per image at shape (1, 3, 1088, 1920)


Processing frames: 2it [00:32, 15.49s/it]


WARNING ⚠️ imgsz=[1080, 1920] must be multiple of max stride 32, updating to [1088, 1920]


Processing frames: 2it [00:37, 18.96s/it]


KeyboardInterrupt: 